In [1]:
from os import path, makedirs
import glob
import json
from sys import exit
import datetime
import time
import re
import urllib3
import certifi
http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED', ca_certs=certifi.where())

In [2]:
with open("../../outputs/compounds_didnt_match_pubchem.tsv", "w") as o:
    o.write("")
def update_compounds(data):
    mibig_id = data["general_params"]["mibig_accession"]
    for compound in data["general_params"]["compounds"]:
        if "pubchem_id" in compound:
            pubchem_id = compound["pubchem_id"]
            pubchem_url = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{}/property/IsomericSMILES,MolecularFormula,MonoisotopicMass/JSON".format(pubchem_id)
            time.sleep(0.2)
            resp = http.request('GET', pubchem_url)
            result = json.loads(resp.data.decode('utf-8'))
            if "PropertyTable" in result:
                resprop = result["PropertyTable"]["Properties"][0]
                if "mol_mass" in compound and resprop["MonoisotopicMass"] != compound["mol_mass"]:
                    with open("../../outputs/compounds_didnt_match_pubchem.tsv", "a") as o:
                        o.write("{}\tmol_mass\t{}\t{}\t{}\t{}\n".format(mibig_id, compound["compound"], pubchem_id, compound["mol_mass"], resprop["MonoisotopicMass"]))
                compound["mol_mass"] = resprop["MonoisotopicMass"]
                if "molecular_formula" in compound and resprop["MolecularFormula"] != compound["molecular_formula"]:
                    with open("../../outputs/compounds_didnt_match_pubchem.tsv", "a") as o:
                        o.write("{}\tmolecular_formula\t{}\t{}\t{}\t{}\n".format(mibig_id, compound["compound"], pubchem_id, compound["molecular_formula"], resprop["MolecularFormula"]))
                compound["molecular_formula"] = resprop["MolecularFormula"]
                if "chem_struct" in compound and resprop["IsomericSMILES"] != compound["chem_struct"]:
                    with open("../../outputs/compounds_didnt_match_pubchem.tsv", "a") as o:
                        o.write("{}\tchem_struct\t{}\t{}\t{}\t{}\n".format(mibig_id, compound["compound"], pubchem_id, compound["chem_struct"], resprop["IsomericSMILES"]))
                compound["chem_struct"] = resprop["IsomericSMILES"]
                ## get synonyms
                url = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{}/synonyms/TXT".format(pubchem_id)
                time.sleep(0.2)
                resp = http.request('GET', url)
                synonyms = [syn for syn in resp.data.decode('utf-8').split("\n") if syn != compound["compound"]]
                if "chem_synonyms" in compound and set(synonyms) != set(compound["chem_synonyms"]):
                    with open("../../outputs/compounds_didnt_match_pubchem.tsv", "a") as o:
                        o.write("{}\tsynonyms\t{}\t{}\t{}\t{}\n".format(mibig_id, compound["compound"], pubchem_id, ";".join(compound["chem_synonyms"]), ";".join(synonyms)))
                compound["chem_synonyms"] = synonyms
        ## todo: other databases
    pass

In [3]:
def query_pubchem_id(compound_name):
    url = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{}/cids/JSON?list_return=flat".format(compound_name.replace(" ", "%20"))
    time.sleep(0.2)
    resp = http.request('GET', url)
    result = json.loads(resp.data.decode('utf-8'))
    if "IdentifierList" in result:
        for cid in result["IdentifierList"]["CID"]:
            # get the one with exact compound name (i.e. no synonym version)
            url = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{}/synonyms/TXT".format(cid)
            time.sleep(0.2)
            resp = http.request('GET', url)
            results = resp.data.decode('utf-8').split("\n")
            if results[0].lower().replace(" ", "") == compound_name.lower().replace(" ", ""):
                return str(cid)
    return None
            

In [4]:
input_path = "../../preprocessed/json_2.0_genefinding/"
output_folder = "../../outputs/json_2.0/"

if not path.exists("../../outputs/json_2.0/"):
    makedirs("../../outputs/json_2.0/")

with open("../../preprocessed/compounds_without_structures.tsv", "w") as o:
    o.write("")
for json_path in sorted(glob.glob(path.join(input_path, "BGC*.json"))):
    print("Fetching compounds data for {}..".format(json_path))
    with open(json_path, "r") as json_file:
        data = json.load(json_file)
        bgc_id = path.basename(json_path).split(".")[0]
        for compound in data["general_params"]["compounds"]:
            if "chem_struct" not in compound or len(compound["chem_struct"]) < 1:
                if ("pubchem_id" not in compound and
                    "chebi_id" not in compound and
                    "chembl_id" not in compound and
                    "chemspider_id" not in compound):
                    qpubchem_id = query_pubchem_id(compound["compound"])
                    if qpubchem_id != None:
                        compound["pubchem_id"] = qpubchem_id
                    with open("../../preprocessed/compounds_without_structures.tsv", "a") as o:
                        o.write("{}\t{}\t{}\n".format(bgc_id, compound["compound"], qpubchem_id))
        update_compounds(data)
        with open(path.join(output_folder, "{}.json".format(bgc_id)), "w") as o:
            o.write(json.dumps(data, indent=4, separators=(',', ': ')))
print("All data fetched!")

Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0000001.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0000002.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0000003.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0000004.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0000005.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0000006.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0000007.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0000008.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0000009.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0000010.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0000011.json..
Fetching compounds data for ../../preprocessed/json_2.

Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0000098.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0000099.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0000100.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0000101.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0000102.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0000103.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0000104.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0000105.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0000106.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0000107.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0000108.json..
Fetching compounds data for ../../preprocessed/json_2.

Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0000198.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0000199.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0000200.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0000201.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0000202.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0000203.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0000204.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0000205.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0000206.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0000207.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0000208.json..
Fetching compounds data for ../../preprocessed/json_2.

Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0000294.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0000295.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0000296.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0000297.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0000298.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0000299.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0000300.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0000301.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0000302.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0000303.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0000304.json..
Fetching compounds data for ../../preprocessed/json_2.

Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0000393.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0000394.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0000395.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0000396.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0000397.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0000398.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0000399.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0000400.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0000401.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0000402.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0000403.json..
Fetching compounds data for ../../preprocessed/json_2.

...

Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0001365.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0001366.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0001367.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0001368.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0001369.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0001370.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0001371.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0001372.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0001373.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0001374.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0001375.json..
Fetching compounds data for ../../preprocessed/json_2.

Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0001461.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0001462.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0001463.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0001464.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0001465.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0001466.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0001467.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0001468.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0001469.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0001470.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0001471.json..
Fetching compounds data for ../../preprocessed/json_2.

Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0001560.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0001561.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0001562.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0001563.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0001564.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0001565.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0001566.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0001567.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0001568.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0001569.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0001570.json..
Fetching compounds data for ../../preprocessed/json_2.

Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0001656.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0001657.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0001658.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0001659.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0001660.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0001661.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0001662.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0001663.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0001664.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0001665.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0001666.json..
Fetching compounds data for ../../preprocessed/json_2.

Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0001753.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0001754.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0001755.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0001756.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0001757.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0001758.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0001759.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0001760.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0001761.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0001762.json..
Fetching compounds data for ../../preprocessed/json_2.0_genefinding/BGC0001763.json..
Fetching compounds data for ../../preprocessed/json_2.